In [115]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
from os.path import join as pjoin
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pingouin as pg

sys.path.append("../..")
import circletrack_behavior as ctb
import circletrack_neural as ctn
import plotting_functions as pf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
## Set parameters
behavior_path = '../../../MultiCon_AfterHours/'
fig_path = '../../../MultiCon_AfterHours/MultiCon_EEG1/intermediate_figures/'
num_experiments = 2 + 1 ## change first value since np.arange is not inclusive
experiment_list = ['MultiCon_EEG{}'.format(x) for x in np.arange(1, num_experiments)]
session_dict = {'mc_EEG1_01': ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5',
                               'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5'],
                'mc_EEG1_02': ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5',
                               'C1', 'C2', 'C3', 'C4', 'C5', 'A2_1', 'A2_2', 'A2_3', 'A2_4', 'A2_5'],
                'mc_EEG2_02': ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5',
                               'C1', 'C2', 'C3', 'C4', 'C5', 'A2_1', 'A2_2', 'A2_3', 'A2_4', 'A2_5']}

In [141]:
## Calculate lick accuracy, dprime metrics during non-probe
experiment_pc = {}
experiment_pc_trial = {}
experiment_sdt = {}
for experiment in experiment_list:
    bpath = pjoin(behavior_path, experiment)
    bpath = pjoin(bpath, 'output/behav')
    mouse_pc = {}
    mouse_pc_trial = {}
    mouse_sdt = {}
    for mouse in os.listdir(bpath):
        path = pjoin(bpath, mouse)
        percent_correct = {}
        percent_correct_trials = {}
        sdt_metrics = {}
        for session in np.arange(0, len(session_dict[mouse])):
            behav = pd.read_feather(pjoin(path, '{}_{}.feat'.format(mouse, session_dict[mouse][session])))
            behav_nonprobe = behav.loc[~behav['probe']]
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            forward_trials, reverse_trials = ctb.get_forward_reverse_trials(behav)
            forward_df = behav.loc[(behav['trials'].isin(forward_trials))]
            forward_df = forward_df.loc[~forward_df['probe']]
            ## Percent correct licking
            pc = ctb.lick_accuracy(behav_nonprobe, port_one=reward_one, port_two=reward_two, by_trials=False)
            pc_trials = ctb.lick_accuracy(forward_df, port_one=reward_one, port_two=reward_two, by_trials=True)
            percent_correct[session] = pc
            percent_correct_trials[session] = pc_trials
            ## dprime, correct rejection, hit rate
            metrics = ctb.dprime_metrics(forward_df, reward_one, reward_two, reward_index='one', forward_reverse='forward')
            sdt_metrics[session] = metrics
        mouse_pc[mouse] = percent_correct
        mouse_pc_trial[mouse] = percent_correct_trials
        mouse_sdt[mouse] = sdt_metrics
    experiment_pc[experiment] = mouse_pc
    experiment_pc_trial[experiment] = mouse_pc_trial
    experiment_sdt[experiment] = mouse_sdt

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../circletrack_behavior.py:1010: RuntimeWarning:

invalid value encountered in long_scalars

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../circletrack_behavior.py:1010: RuntimeWarning:

invalid value encountered in long_scalars

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../circletrack_behavior.py:1010: RuntimeWarning:

invalid value encountered in long_scalars

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../circletrack_behavior.py:1010: RuntimeWarning:

invalid value encountered in long_scalars

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../circletrack_behavior.py:1010: RuntimeWarning:

invalid value encountered in long_scalars

/media/caishuman/csstorage3/Austin/Circl

In [152]:
## Percent correct data non_probe
accuracy = pd.DataFrame()
for experiment in experiment_pc:
    for mouse in experiment_pc[experiment]:
        if mouse == 'mc_EEG2_01':
            next
        else:
            df = pd.DataFrame(experiment_pc[experiment][mouse], index=[0]).T.rename(columns={0: 'percent_correct'})
            df.insert(0, 'mouse', mouse)
            df.insert(1, 'day', [x for x in np.arange(1, 21)])
            accuracy = pd.concat([accuracy, df], axis=0, ignore_index=True)

fig = pf.plot_behavior_across_days(accuracy, x_var='day', y_var='percent_correct', groupby_var=['day'], titles=['Lick Accuracy'],
                                   x_title='Day', y_title='Percent Correct (%)', template='simple_white', expert_line=False)
fig.update_xaxes(dtick=2) 
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy.png'))

In [143]:
## Get performance drop for each mouse
performance_drop = ctb.performance_drop(accuracy, day_list=[5, 10, 15], replace=True)
performance_drop = performance_drop.replace({5: 'A to B', 10: 'B to C', 15: 'C to D'})
avg_drop = performance_drop.groupby(['day'], as_index=False).agg({'drop': ['mean', 'sem']})
avg_drop_df = avg_drop['drop']
avg_drop_df.insert(0, 'day', avg_drop['day'])
## Plot fig
fig = pf.custom_graph_template(x_title='', y_title='Performance Difference (%)')
fig.add_trace(go.Bar(x=avg_drop_df['day'], y=avg_drop_df['mean'],
                     error_y=dict(type = 'data', array=avg_drop_df['sem'], thickness = 2.5, width = 10),
                     marker_color='darkgrey', marker_line_color='black',
                     marker_line_width=2, showlegend=False, width=[0.5, 0.5, 0.5]))
fig.add_trace(go.Scatter(x=performance_drop['day'], y=performance_drop['drop'], mode='markers', 
                            marker_color='black', showlegend=False))
fig.show()
fig.write_image(pjoin(fig_path, 'performance_drop.png'))

In [144]:
pg.homoscedasticity(data=performance_drop, dv='drop', group='day')

,W,pval,equal_var
levene,0.361462,0.710852,True


In [145]:
performance_drop.anova(dv='drop', between='day', detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,day,2276.150957,2,1138.075479,6.818438,0.028526,0.694452
1,Within,1001.468738,6,166.911456,NaN,NaN,NaN


In [146]:
performance_drop.pairwise_tukey(dv='drop', between='day')

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,A to B,B to C,46.883706,35.481388,11.402318,10.548664,1.080925,0.558701,0.706057
1,A to B,C to D,46.883706,8.924744,37.958962,10.548664,3.598462,0.026460,2.350505
2,B to C,C to D,35.481388,8.924744,26.556644,10.548664,2.517536,0.099823,1.644448


In [37]:
## Calculate percent correct during probe
experiment_pc_probe = {}
for experiment in experiment_list:
    bpath = pjoin(behavior_path, experiment)
    bpath = pjoin(bpath, 'output/behav')
    mouse_pc = {}
    for mouse in os.listdir(bpath):
        path = pjoin(bpath, mouse)
        percent_correct = {}
        for session in np.arange(0, len(session_dict[mouse])):
            behav = pd.read_feather(pjoin(path, '{}_{}.feat'.format(mouse, session_dict[mouse][session])))
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            probe_behav = behav.loc[behav['probe']]
            pc = ctb.probe_lick_accuracy(probe_behav, port_one=reward_one, port_two=reward_two)
            percent_correct[session] = pc
        mouse_pc[mouse] = percent_correct
    experiment_pc_probe[experiment] = mouse_pc

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../circletrack_behavior.py:967: RuntimeWarning:

invalid value encountered in long_scalars

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../circletrack_behavior.py:967: RuntimeWarning:

invalid value encountered in long_scalars

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../circletrack_behavior.py:967: RuntimeWarning:

invalid value encountered in long_scalars



In [151]:
## Percent correct probe
accuracy_probe = pd.DataFrame()
for experiment in experiment_pc_probe:
    for mouse in experiment_pc_probe[experiment]:
        if mouse == 'mc_EEG2_01':
            next
        else:
            df = pd.DataFrame(experiment_pc_probe[experiment][mouse], index=[0]).T.rename(columns={0: 'percent_correct'})
            df.insert(0, 'mouse', mouse)
            df.insert(1, 'day', [x for x in np.arange(1, 21)])
            accuracy_probe = pd.concat([accuracy_probe, df], axis=0, ignore_index=True)
accuracy_probe = accuracy_probe[accuracy_probe['day'] > 1]
fig = pf.plot_behavior_across_days(accuracy_probe, x_var='day', y_var='percent_correct', groupby_var=['day'], titles=['Lick Accuracy During Probe'],
                                   x_title='Day', y_title='Percent Correct (%)', template='simple_white', expert_line=False)
fig.update_xaxes(dtick=2)                                   
fig.show()
fig.write_image(pjoin(fig_path, 'probe_accuracy.png'))

In [39]:
## d' across days
dprime_df = pd.DataFrame()
for experiment in experiment_pc:
    for mouse in experiment_pc[experiment]:
        if mouse == 'mc_EEG2_01':
            next
        else:
            df = pd.DataFrame(experiment_sdt[experiment][mouse]).T
            d = []
            for idx in np.arange(0, len(df['dprime'])):
                d.append(np.nanmean(df['dprime'][idx]))
            dprime = pd.DataFrame(d).rename(columns={0: 'dprime'})
            dprime.insert(0, 'mouse', mouse)
            dprime.insert(1, 'day', [x for x in np.arange(1, 21)])
            dprime_df = pd.concat([dprime_df, dprime], axis=0, ignore_index=True)

fig = pf.plot_behavior_across_days(dprime_df, x_var='day', y_var='dprime', groupby_var=['day'], titles=[''],
                                   x_title='Day', y_title="d'", template='simple_white')
fig.show()

/tmp/ipykernel_27202/17010159.py:11: RuntimeWarning:

Mean of empty slice



In [148]:
fig = pf.plot_lick_raster(behav, titles=['Lick Raster'], x_title='Linearized Position (cm)', y_title='Trial')
fig.show()

In [ ]:
fig = pf.plot_circle_position(behav, title='', x_title='Position (x)', y_title='Position (y)')
fig.show()

In [131]:
## Plot for performance drop vs percent overlap
## This is not how I will do this in the future, just need this for proposal
## Values come from circletrack_overlap.ipynb
performance_dict = {'mc_EEG1_01': {'drop': [29.45, 26.92, 0], 'overlap': [27.55, 53.61, 52.48]},
                    'mc_EEG1_02': {'drop': [70.52, 27.5], 'overlap': [40.62, 51.94]}}
df = pd.DataFrame(performance_dict)
y_data = np.concatenate((df.loc['drop', 'mc_EEG1_01'], df.loc['drop', 'mc_EEG1_02']))
x_data = np.concatenate((df.loc['overlap', 'mc_EEG1_01'], df.loc['overlap', 'mc_EEG1_02']))
lm = pg.linear_regression(X=x_data, y=y_data, as_dataframe=False)
lm_x = []
for n in np.arange(len(lm['X'])):
    lm_x.append(lm['X'][n][1])
colors = ['darkgrey', 'turquoise']
fig = pf.custom_graph_template(x_title='Neuronal Overlap (%)', y_title='Performance Drop (%)')
for idx, mouse in enumerate(df.columns):    
    fig.add_trace(go.Scatter(x=df.loc['overlap', mouse], y=df.loc['drop', mouse], 
                             mode='markers', marker=dict(color=colors[idx]), name=mouse))
fig.add_trace(go.Scatter(x=lm_x, y=lm['pred'], mode='lines', line_color='turquoise', showlegend=False))
fig.show()
fig.write_image(pjoin(fig_path, 'perf_drop_vs_overlap.png'))

In [134]:
pg.corr(x=x_data, y=y_data, method='spearman')

,n,r,CI95%,p-val,power
spearman,5,-0.8,"[-0.99, 0.28]",0.104088,0.405232


## Testing for place cell code below.

In [3]:
pc_path = '../../../MultiCon_AfterHours/MultiCon_EEG1/minian_result/mc_EEG1_01/minian/'
behav = pd.read_feather(os.path.abspath(pjoin(behavior_path, 'MultiCon_EEG1/output/behav/mc_EEG1_01/mc_EEG1_01_A1.feat')))
# spikes = ctn.open_minian(pc_path)['S']

In [4]:
behav

,unix,frame,t,x,y,a_pos,lick_port,water,animal,session,cohort,trials,lin_position,reward_one,reward_two,probe
0,1.666192e+09,1,0.011516,314.0,47.0,88.0,-1,False,mc_EEG1_01,A1,cohort0,0,1.535890,3,7,True
1,1.666192e+09,2,0.039580,314.0,47.0,88.0,-1,False,mc_EEG1_01,A1,cohort0,0,1.535890,3,7,True
2,1.666192e+09,3,0.061317,315.0,48.0,88.0,-1,False,mc_EEG1_01,A1,cohort0,0,1.535890,3,7,True
3,1.666192e+09,5,0.107424,317.0,48.0,87.0,-1,False,mc_EEG1_01,A1,cohort0,0,1.518436,3,7,True
4,1.666192e+09,6,0.129446,319.0,48.0,86.0,-1,False,mc_EEG1_01,A1,cohort0,0,1.500983,3,7,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,1.666193e+09,51146,1199.840613,501.0,177.0,15.0,-1,False,mc_EEG1_01,A1,cohort0,12,0.261799,3,7,False
35996,1.666193e+09,51147,1199.863639,500.0,174.0,16.0,-1,False,mc_EEG1_01,A1,cohort0,12,0.279253,3,7,False
35997,1.666193e+09,51148,1199.888682,499.0,171.0,17.0,-1,False,mc_EEG1_01,A1,cohort0,12,0.296706,3,7,False
35998,1.666193e+09,51150,1199.937236,498.0,169.0,17.0,-1,False,mc_EEG1_01,A1,cohort0,12,0.296706,3,7,False


In [7]:
reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
probe_behav = behav.loc[behav['probe']]
pc = ctb.lick_accuracy(probe_behav, port_one=reward_one, port_two=reward_two, by_trials=False)
pc

50.0

In [ ]:
def align_start_times(act, behav):
    start_idx = np.abs(behav.loc[0, 'unix'] - act['unix'].values).argmin()
    return act[:, start_idx:].rename('S_shifted')

In [ ]:
test = align_start_times(act=spikes, behav=behav)
test

In [ ]:
end_time = np.abs(behav.loc[35999, 'unix'] - spikes['unix'].values).argmin()
end_time

In [ ]:
behav.loc[35999, 'unix']

In [ ]:
spikes['unix'].values[17530]

## Behavior preprocessing code:

In [ ]:
import re
cohort_number = 'cohort0'
mouse = 'mc_EEG1_01'
session = 'A2'
sampling_rate = 1/30 ## started keeping minian output at 1/30
frame_count = 20 * 60 / sampling_rate ## session length in minutes x 60s per minute / sampling rate
## 2022_10_21/11_11_20
## 2022_10_22/18_14_11
circle_track = pd.read_csv('../../../MultiCon_AfterHours/MultiCon_EEG1/circletrack_data/data/mc_EEG1_01/2022_10_22/18_14_11/circle_track.csv')
rewards = circle_track.loc[circle_track['event'] == 'initializing', 'data'].tolist()
for idx in np.arange(0, len(circle_track['event'])):
    if 'probe' in circle_track['event'][idx]:
        probe_end = float(re.search('probe length: ([0-9]+)', circle_track['event'][idx])[1])
    else:
        next
circle_track = ctb.crop_data(circle_track)
unix_start =  circle_track.loc[circle_track['event'] == 'START', 'timestamp'].values[0]
time_vector = np.arange(unix_start, (frame_count * sampling_rate + unix_start), sampling_rate)
circle_track["frame"] = np.arange(len(circle_track))
data_out = circle_track[circle_track["event"] == "LOCATION"].copy().reset_index(drop=True)
arg_mins = [np.abs(data_out['timestamp'] - t).argmin() for t in time_vector] ## resample to sampling freq of time_vector
data_out = data_out.loc[arg_mins, :].reset_index(drop=True)
events = circle_track[circle_track["event"] != "LOCATION"].copy().reset_index(drop=True)
data_out['t'] = (pd.to_numeric(data_out['timestamp'] - unix_start))
data_out[["x", "y", "a_pos"]] = (
    data_out["data"]
    .apply(
        lambda d: pd.Series(
            re.search(
                r"X(?P<x>[0-9]+)Y(?P<y>[0-9]+)A(?P<ang>[0-9]+)", d
            ).groupdict()
        )
    )
    .astype(float)
)
data_out["lick_port"] = -1
data_out["water"] = False
for _, row in events.iterrows():
    ts = row["timestamp"]
    idx = np.argmin(np.abs(data_out["timestamp"] - ts))
    try:
        port = int(row["data"][-1])
    except TypeError:
        continue
    data_out.loc[idx, "lick_port"] = port
    if row["event"] == "REWARD":
        data_out.loc[idx, "water"] = True
data_out[["animal", "session", "cohort"]] = mouse, session, cohort_number
# data_out["trials"] = ctb.get_trials(
#     data_out, shift_factor=0, angle_type="radians", counterclockwise=True
# )
# data_out["lin_position"] = ctb.linearize_trajectory(
#     data_out, angle_type="radians", shift_factor=0
# )
# data_out[['reward_one', 'reward_two']] = int(rewards[0][-1]), int(rewards[1][-1])
# data_out = (
#     data_out.drop(columns=["event", "data"])
#     .rename(columns={"timestamp": "unix"})
#     .reset_index(drop=True)
# )
# data_out['probe'] = data_out['t'] < probe_end

In [ ]:
data_out

In [ ]:
## Linearize then bin position into one of 8 bins
counterclockwise = True
angle_type = 'radians'
shift_factor = 0
df = data_out
position = ctb.linearize_trajectory(df, shift_factor = shift_factor, angle_type = angle_type)
binned_position = ctb.bin_linearized_position(position, angle_type = angle_type)
bins = np.unique(binned_position)
## For each bin, get timestamps when the mouse was in that bin
indices = [np.where(binned_position == this_bin)[0] for this_bin in bins]
if counterclockwise:  ## reverse the order of the bins.
    indices = indices[::-1]
## Preallocate trial vector
trials = np.full(binned_position.shape, np.nan)
trial_start = 0
last_idx = 0
# We need to loop through bins rather than simply looking for border crossings because a mouse can backtrack, which we wouldn't want to count.
# For a large number of trials...
for trial_number in range(1):
    ## For each bin...
    for this_bin in indices:
        ## Find the first timestamp that comes after the first timestamp in the last bin for that trial. 
        ## Argmax is supposedly faster than np.where.
        last_idx = this_bin[np.argmax(this_bin > last_idx)]
    ## After looping through all the bins, remember the last timestamp where there was a bin transition.
    trial_end = last_idx
    ## If the slice still has all NaNs, label it with the trial number.
    if np.all(np.isnan(trials[trial_start:trial_end])):
        trials[trial_start:trial_end] = trial_number
    ## If not, finish up and exit the loop.
    else:
        trials[np.isnan(trials)] = trial_number - 1
        break
    ## The start of the next trial is the end of the last.
    trial_start = trial_end
trials.astype(int)